# Step 4.1 — Data parsing & cleaning (TSLA daily)

**Goal.** Load daily OHLC data for **TSLA**, standardize it, and produce a clean **Close** price series ready for log-returns and calibration.

**Input (file):** `GeometricBrownianMotion\data\daily\tsla.us.txt`
Format (no header, comma-separated):
`<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>,<OPENINT>`

* We’ll parse `DATE`+`TIME` → a proper datetime index.
* We’ll keep **CLOSE** as the working price (if you have **Adjusted Close** elsewhere, prefer that).
* Period is already **daily** (`PER = D`), so the time step is

  $$
  \Delta t = \frac{1}{252}\ \text{(trading years)}.
  $$

**What we produce now:** a tidy DataFrame indexed by date with columns
`Close, Open, High, Low, Volume, OpenInt`.

**Minimal formulas used later (recap; see earlier notebooks for derivations):**

* **Log-returns:** $r_t = \ln S_t - \ln S_{t-1}$.
* **Estimators (from Step 2):**

  $$
  \hat\sigma = \sqrt{\frac{\operatorname{Var}(r)}{\Delta t}},\qquad
  \hat\mu = \frac{\overline{r}}{\Delta t} + \tfrac12\,\hat\sigma^{2}.
  $$
* **Exact GBM step (from Step 3):**

  $$
  S_{k+1}=S_k\exp\!\Big((\mu-\tfrac12\sigma^2)\Delta t+\sigma\sqrt{\Delta t}\,Z_k\Big),\quad Z_k\sim\mathcal N(0,1).
  $$

**Sanity checks to remember:**

* Prices must be **positive** (logs need $S_t>0$).
* Dates strictly increasing; **no duplicates** per day.
* Handle or flag **missing Close**; if not using **Adjusted Close**, note that splits/dividends can create large one-day moves (we’ll flag outliers later).

In [ ]:
import pandas as pd
import numpy as np

# ---- Settings ----
CSV_PATH = r"./data/daily/tsla.us.txt"  # raw string for backslashes on Windows
TRADING_DAYS_PER_YEAR = 252
DELTA_T = 1.0 / TRADING_DAYS_PER_YEAR



# ---- Load & clean a single-file (TSLA) in your format ----
def load_single_ohlc_file(csv_path: str):
    cols = ["TICKER","PER","DATE","TIME","OPEN","HIGH","LOW","CLOSE","VOL","OPENINT"]
    df = pd.read_csv(csv_path, header=None, names=cols)

    # Keep daily rows (PER == 'D'), drop rows without CLOSE
    df = df[df["PER"].astype(str).str.upper().eq("D")].copy()
    df = df.dropna(subset=["CLOSE"])

    # Build datetime index from DATE (YYYYMMDD) + TIME (HHMMSS)
    dt_str = df["DATE"].astype(str).str.zfill(8) + df["TIME"].astype(str).str.zfill(6)
    df["DateTime"] = pd.to_datetime(dt_str, format="%Y%m%d%H%M%S", errors="coerce")

    # Standardize column names (lowercase), set index, sort, drop duplicates
    df = df.rename(columns={
        "TICKER":"ticker","PER":"per","OPEN":"open","HIGH":"high",
        "LOW":"low","CLOSE":"close","VOL":"volume","OPENINT":"openint"
    })
    df = df.dropna(subset=["DateTime"]).set_index("DateTime").sort_index()
    df = df[~df.index.duplicated(keep="last")]

    # Cast numerics and keep typical columns
    for c in ["open","high","low","close","volume","openint"]:
        if c in df:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # Enforce positive close, drop remaining NaNs in close
    df = df[df["close"] > 0].dropna(subset=["close"])

    # Final tidy frame
    out = df[["close","open","high","low","volume","openint"]].copy()
    out.rename(columns=str.capitalize, inplace=True)  # Close, Open, High, Low, Volume, Openint
    return out

tsla = load_single_ohlc_file(CSV_PATH)

# ---- Quick sanity report ----
def sanity_report(g: pd.DataFrame):
    return pd.DataFrame({
        "rows": [len(g)],
        "start": [g.index.min()],
        "end": [g.index.max()],
        "missing_close": [g["Close"].isna().sum()],
        "nonpos_close": [(g["Close"] <= 0).sum()]
    })

rep = sanity_report(tsla)
display(rep)
display(tsla.head(3))
display(tsla.tail(3))

FileNotFoundError: [Errno 2] No such file or directory: 'GeometricBrownianMotion/data/daily/tsla.us.txt'